<a href="https://colab.research.google.com/github/MrKkondae/practiceLLM/blob/main/miniLLM_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 라이브러리 설치 및 환경 설정

In [ ]:
!pip install transformers datasets accelerate --upgrade

import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

 2. KoGPT2 모델 및 토크나이저 로드

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>'
)

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

3. 파인튜닝용 대화 데이터셋 정의

In [ ]:
# 간단한 사용자-챗봇 대화 5쌍
raw_data = [
    {"input": "안녕!", "output": "안녕하세요! 무엇을 도와드릴까요?"},
    {"input": "오늘 날씨 어때?", "output": "맑고 화창한 날씨예요. 나들이 가기 좋겠네요!"},
    {"input": "영화 추천해줘", "output": "최근 인기 있는 영화는 '범죄도시4'예요. 액션 좋아하시나요?"},
    {"input": "배고파", "output": "식사 시간인가 봐요. 따뜻한 밥 한 끼 어떠세요?"},
    {"input": "잘자", "output": "좋은 꿈 꾸세요. 내일 봬요!"},
    {"input": "안녕! 오늘 날씨 어때?", "output": "안녕하세요!, 오늘은 많이 덥네요. 잠자기 좋은 날씨예요!"}
]

# GPT 모델에 적합한 포맷으로 변환: "사용자: 질문\n챗봇: 답변"
# 프롬프트 구성 함수 수정
def build_prompt(example):
    return f"사용자: {example['input']}\n챗봇: {example['output']} </s>"

# 데이터셋 생성
prompt_data = [{"text": build_prompt(item)} for item in raw_data]
dataset = Dataset.from_list(prompt_data)


4. 데이터셋 토큰화

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# ✨ text 컬럼 제거 + tokenizer 출력 유지
tokenized_dataset = dataset.map(tokenize_function, remove_columns=["text"])


Map:   0%|          | 0/6 [00:00<?, ? examples/s]

 5. 학습 설정 및 Trainer 구성

In [ ]:
training_args = TrainingArguments(
    output_dir="./kogpt2-finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    logging_steps=5,
    save_steps=10,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)


🚀 6. 파인튜닝 실행

In [ ]:
trainer.train()

Step,Training Loss
5,0.902300
10,0.331100
15,0.193600


TrainOutput(global_step=15, training_loss=0.4756725589434306, metrics={'train_runtime': 205.8884, 'train_samples_per_second': 0.146, 'train_steps_per_second': 0.073, 'total_flos': 1959690240000.0, 'train_loss': 0.4756725589434306, 'epoch': 5.0})

✅ 7. 파인튜닝된 모델로 응답 테스트

In [ ]:
def chat_kogpt2(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to(model.device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response[len(prompt):].strip()



# 대화 루프 시작
print("🗨️ KoGPT2 챗봇입니다. '종료'라고 입력하면 끝나요.")
while True:
    user_input = input("👤 사용자: ")
    if user_input.strip() == "종료":
        print("🤖 챗봇: 안녕히 가세요!")
        break
    full_prompt = f"사용자: {user_input}\n챗봇:"
    response = chat_kogpt2(full_prompt)
    print(f"🤖 챗봇: {response}")


🗨️ KoGPT2 챗봇입니다. '종료'라고 입력하면 끝나요.
👤 사용자: 안녕 오늘날씨 어때?
🤖 챗봇: 안녕하세요! 무엇을 도와드릴까요?
👤 사용자: 안녕! 오늘 날씨 어때?
🤖 챗봇: 안녕하세요!
👤 사용자: 배고파
🤖 챗봇: 식사 시간인가 봐요.
챗봇: 식사 시간인가 봐요. 따뜻한 밥 한 끼 어떠세요?


KeyboardInterrupt: Interrupted by user

In [ ]:
print(tokenized_dataset)
print(tokenized_dataset[0])
print(type(tokenized_dataset[0]['input_ids']))

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5
})
{'input_ids': [33073, 401, 25906, 36500, 8352, 7659, 401, 25906, 8702, 7801, 8084, 376, 22375, 14355, 7281, 7481, 6969, 8084, 406, 739, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1